<a href="https://colab.research.google.com/github/mertcan-basut/nlp/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai

!pip install -q langchain langchain-openai

!pip install -q python-dotenv

In [2]:
!echo "OPENAI_API_KEY=editthis" > .env

In [24]:
# a framework for developing LM powered applications
from langchain_openai import ChatOpenAI # models
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate # input prompts
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.output_parsers import StructuredOutputParser # output parsers
from langchain.output_parsers import ResponseSchema
from langchain.chains import ConversationChain # memory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryBufferMemory

import openai # direct API calls to OpenAI

import json
import os
# https://platform.openai.com/api-keys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

## Models, Prompts, Parsers

In [ ]:
text = """\
This leaf blower is pretty amazing.  It has four settings: \
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.\
"""

system_message_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.
"""

human_message_template = """\
text: {text}
"""

format_instructions_template = """\
Format the output as JSON with the following keys:
gift
delivery_days
price_value
"""

In [ ]:
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

get_completion("Hi!")

'Hello! How can I assist you today?'

In [ ]:
def format_prompt(system_message, human_message, format_instructions):
  return f"""\
{system_message}

{human_message}

{format_instructions}

"""

system_message_prompt = system_message_template.format()
human_message_prompt = human_message_template.format(text=text)
format_instructions_prompt = format_instructions_template.format()
prompt = format_prompt(system_message_prompt, human_message_prompt, format_instructions_prompt)
print(prompt)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.


text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


Format the output as JSON with the following keys:
gift
delivery_days
price_value





In [ ]:
response = get_completion(prompt)
print(response)

output_dict = json.loads(response)
output_dict.get('delivery_days')

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


2

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

chat([HumanMessage(content="Hi!")]).content

'Hello! How can I assist you today?'

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)

gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days", description="How many days did it take for the product to arrive? If this information is not found, output -1.")
price_value_schema = ResponseSchema(name="price_value", description="Extract any sentences about the value or price, and output them as a comma separated Python list.")
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
format_instructions_prompt = SystemMessage(content=format_instructions)

prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, format_instructions_prompt])
prompt = prompt_template.format_messages(text=text)
for message in prompt: print(message.content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "`

In [ ]:
response = chat(prompt).content
print(response)

output_dict = output_parser.parse(response)
output_dict.get('delivery_days')

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


2

## Memory

In [4]:
llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)

In [5]:
print(conversation.predict(input="Hi, my name is Mert."))
print(conversation.predict(input="What is 1+1?"))
print(conversation.predict(input="What is my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Mert.
AI:

> Finished chain.
Hello Mert! It's nice to meet you. How can I assist you today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.
1 + 1 equals 2. Is there anything else you would like to know?


> Entering new ConversationChain cha

In [13]:
print(memory.buffer, '\n')
memory.load_memory_variables({})

Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Mert. Is there anything else you would like to know or discuss? 



{'history': "Human: Hi, my name is Mert.\nAI: Hello Mert! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Mert. Is there anything else you would like to know or discuss?"}

In [14]:
# add aditional data to memory
memory.save_context({"input": "Hi!"}, {"output": "What's up?"})
print(memory.buffer)

Human: Hi, my name is Mert.
AI: Hello Mert! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Mert. Is there anything else you would like to know or discuss?
Human: Hi!
AI: What's up?


In [17]:
memory = ConversationBufferWindowMemory(k=1) # only keeps most recent `k` conversations

memory.save_context({"input": "Hi!"}, {"output": "What's up?"})
memory.save_context({"input": "Not much, just hanging."}, {"output": "Cool."})

print(memory.buffer)

Human: Not much, just hanging.
AI: Cool.


In [23]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30) # chops off the earlier parts of the conversation to not exceed the token limit dependent on the LLM because usually cost is determined by number of tokens

memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

print(memory.buffer)

AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [28]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=30) # uses the specified LLM to summarize the entire chat history which exceeds the specified number of tokens

memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

print(memory.load_memory_variables({})['history'])
# `System` is not official OpenAI system message!

System: The human expresses surprise at the AI's positive view of artificial intelligence. The AI responds with "Amazing!" and the human asks about backpropagation.
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!
